In [1]:
import warnings
from typing import List

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from autumn.core.project import get_project
from autumn.settings import Region, Models
from autumn.core import inputs
from autumn.core.utils.display import pretty_print
from autumn.core.inputs.tb_camau import queries
from autumn.core.inputs import get_death_rates_by_agegroup
import pathlib

In [2]:
warnings.filterwarnings("ignore")
pd.options.plotting.backend = "plotly"

In [3]:
region = Region.CAMAU
model = Models.TBD2
p = get_project(model, region, reload=True)
baseline_params = p.param_set.baseline
plots = p.plots
iso3 = baseline_params['country']['iso3']
source_iso3 = baseline_params['age_mixing']['source_iso3']
age_adjust = baseline_params['age_mixing']['age_adjust']
age_string_map = {
    0: "0-4",
    5: "5-14",
    15: "15-34",
    35: "35-49",
    50: "50+",
}

## Population

In [4]:
modelled_age_groups = baseline_params["age_breakpoints"]
print(f"Modelled age groups are {modelled_age_groups}")

Modelled age groups are [0, 5, 15, 35, 50]


In [5]:
csv_path = pathlib.Path('camau.csv')

In [6]:
pop_df = pd.read_csv(csv_path)
pop_df = pop_df.set_index(['year'])
pop_df['population'].plot()

### Birth rate

In [7]:
birth_rates, years = inputs.get_crude_birth_rate("VNM")
birth_rates = [b / 1000.0 for b in birth_rates]  # Birth rates are provided / 1000 population
birth_rates_series = pd.Series(birth_rates, index=years)
br_fig= px.line(birth_rates_series)
br_fig.update_traces(mode='markers+lines')
br_fig.update_layout(
    title="Crude birth rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude birth rate",
    showlegend=False,
)

### Life expectancy

In [8]:
# Currently used in model:
death_rates_from_db = get_death_rates_by_agegroup(modelled_age_groups, "VNM")
death_rates_df = pd.DataFrame(death_rates_from_db[0], index=death_rates_from_db[1])
life_exp_at_age_df = 1.0 / death_rates_df
death_rates_df.plot()
life_exp_at_age_df.plot(log_y=True)

In [9]:
# Query and visualize the crude birth rate from database
years, death_rates = queries.get_camau_death_rate()
death_rates = [b / 1000.0 for b in death_rates]  # Birth rates are provided / 1000 population
death_rates_series = pd.Series(death_rates, index=years)
dr_fig= px.line(death_rates_series)
dr_fig.update_traces(mode='markers+lines')
dr_fig.update_layout(
    title="Crude death rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude death rate",
    showlegend=False,
)

### Mixing matrix

In [10]:
print(f"Target region: {region.upper()} ({iso3})")
print(f"Proxy country: {source_iso3}")
print("Always age-adjusted to target population")

Target region: CAMAU (VNM)
Proxy country: VNM
Always age-adjusted to target population


In [11]:
update_params = {
    'cdr_adjustment': 0.6003452159878636,
    'contact_rate': 0.013414102898074345,
    'gender.adjustments.infection.male': 2.6408657914674176,
    'infect_death_rate_dict.smear_negative': 0.027588310343242016,
    'infect_death_rate_dict.smear_positive': 0.364337776897486,
    'progression_multiplier': 0.11085543897109104,
    'rr_infection_latent': 0.20278196465900813,
    'rr_infection_recovered': 0.21190687223342505,
    'self_recovery_rate_dict.smear_positive': 0.20344728302826143,
    'self_recovery_rate_dict.smear_negative': 0.22723824998716693,
    'start_population_size': 267252.06827576435
}
params = baseline_params.update(update_params,calibration_format=True)
model_0  =  p.build_model(params.to_dict())
model_0.run(params.to_dict())
derived_df_0 = model_0.get_derived_outputs_df()

In [12]:
model_0._stratifications

[Stratification: age, Stratification: organ]

In [13]:
model_0._mixing_matrices

[array([[ 398.43289672,  261.82020387,  643.68286218,  401.62199159,
          356.13449939],
        [ 165.78966683,  881.63067677,  532.84120554,  550.75979227,
          285.62836724],
        [ 231.75164317,  311.38983781,  915.52884268,  673.30894113,
          664.14577066],
        [ 141.94492435,  310.88835505,  786.13676958, 1134.31076003,
          938.03403291],
        [  67.30073632,  170.46333134,  647.30153978, 1018.81243422,
         1763.57657715]])]

In [14]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [15]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Percentage latent"
)
latent_plot.show()

In [16]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notifications",
)
notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
notif_2.update_traces(marker=dict(color="red"))
notif_plot = go.Figure(
    data=notif_1.data + notif_2.data,
)
notif_plot.update_layout(
    title="Modelled vs Data (with James' equation)", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)
notif_plot.show()

In [17]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="prevalence_infectious",
)
notif_1.show()
# notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
# notif_2.update_traces(marker=dict(color="red"))
# notif_plot = go.Figure(
#     data=notif_1.data + notif_2.data,
# )
# notif_plot.update_layout(
#     title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
# )
# notif_plot.show()

In [18]:
model_0.get_derived_outputs_df()

,cumulative_deaths,cumulative_diseased,incidence,incidence_early,incidence_late,notifications,percentage_latent,prevalence_infectious,total_population
1900.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.780449,0.000000,0.374179,2.672521e+05
1900.1,0.000000,0.000000,0.869260,2.322593,0.000094,0.771960,0.000451,0.368910,2.672027e+05
1900.2,0.000000,0.000000,2.204147,5.888167,0.000430,0.768709,0.000872,0.376643,2.671598e+05
1900.3,0.000000,0.000000,2.942511,7.859117,0.001018,0.785924,0.001277,0.390766,2.671234e+05
1900.4,0.000000,0.000000,3.387961,9.047228,0.001789,0.813637,0.001678,0.408458,2.670933e+05
...,...,...,...,...,...,...,...,...,...
2019.6,60425.830487,191075.532611,214.965659,2138.538773,492.389545,1475.647811,29.681892,46.940814,1.223883e+06
2019.7,60486.324072,191339.054380,215.277382,2143.882957,491.334728,1475.434256,29.617596,46.911636,1.224103e+06
2019.8,60546.824722,191603.030314,215.609902,2149.476059,490.283281,1475.470294,29.553511,46.887161,1.224322e+06
2019.9,60607.341600,191867.400651,215.893801,2154.468170,489.235207,1475.714134,29.489590,46.866621,1.224539e+06
